In [1]:
# Referência: https://www.geeksforgeeks.org/python-similarity-metrics-of-strings/
from difflib import SequenceMatcher

def similar(str1, sequencia):
    # str1 = cidade a ser procurada
    # sequencia = conjunto de nomes que serão usados para ver qual mais se aproxima
    maior =  0
    # Para economizar processamento, avalia apenas os que começam com a mesma letra,
    # considerando que o erro de grafia não esteja justamente na letra inicial
    mesma_letra = [g for g in sequencia if g.startswith(str1[0])]
    for i in mesma_letra:
        corr = SequenceMatcher(None, str1, i).ratio()
        # Ignora os que têm menos de 60% de correlação
        # Registra qual nome de cidade mais se aparenta ao que procuramos
        if corr > 0.6 and corr > maior:
            maior = corr
            result = i
    return result

In [2]:
from requests import get
from pandas import read_csv
# Importa as coordenadas poligonais para preencher o mapa com os municípios
response = get('https://raw.githubusercontent.com/jonates/opendata/master/arquivos_geoespaciais/geojs-100-mun.json')
geo_uf = response.json()
# Pega do arquivo CSV apenas as informações que serão usadas
df = read_csv("municipios.csv", usecols=["municipio","mesoregion","name","pop_21", "uf_code"])
# Reduz o dataframe para o estado que queremos
df = df.query("uf_code == 'MG'")
geo_uf['features'] = [f for f in geo_uf['features'] if f["properties"]["id"] in str(df["municipio"].values)]
# Pega os nomes nas coordenadas que não contêm na relação de cidades/população do CSV
faltam = [f["properties"]["name"] for f in geo_uf['features'] if f["properties"]["name"] not in str(df["name"].values)]
# Pega os nomes das cidades mais parecidas no arquivo CSV e substitui para fechar a ligação que pegará as coordenadas
for f in faltam:
        df.at[df.loc[df["name"] == similar(f,df["name"])].index[0],"name"] = f

In [3]:
# from json import dump
# with open('coordenadas.json', 'w') as f:
#    # write the dictionary to the file in JSON format
#    dump(geo_uf, f)

In [4]:
# from json import load
# with open("coordenadas.json", 'r') as file:
#    # Load JSON data into a dictionary
#    geo_uf = load(file)

In [5]:
# Referência: https://www.flai.com.br/jonatas/4-bibliotecas-para-mapas-no-python-altair-bokehfolium-e-plotly/
def exibe_mapa (regiao):  
    # Limpa a saída para não termos vários mapas seguidos
    clear_output(wait=True)    
    # Criando o mapa
    mapa = choropleth_mapbox(
        data_frame = df.query("mesoregion == @regiao"),
        geojson = geo_uf, 
        locations='name', 
        featureidkey='properties.name',
        color='pop_21',
        color_continuous_scale= 'YlOrRd',
        mapbox_style='carto-positron',
        center = {"lat": -18.4038, "lon": -44.4588},
        opacity=1,
        width = 1000,
        zoom=5.5,
        height = 800,
        hover_data={'pop_21':':.', 'name':False},
        hover_name='name',
        labels={'pop_21':'População (2021)'}
        )    
    # Ajustando as margens
    mapa.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

    # Reduzindo a largura das bordas dos municípios
    mapa.update_traces(marker_line_width=0.01)

    # Exibindo o mapa
    mapa.show()

In [6]:
# Pegar dados de API do IBGE para comporem o menu para exibição do mapa por região
api = get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/mg/municipios")
from pandas import json_normalize
cabecalho = json_normalize(api.json())[["nome","microrregiao.mesorregiao.nome","microrregiao.mesorregiao.id"]]
cabecalho.rename(columns={"microrregiao.mesorregiao.nome": "Mesorregião","nome": "Nome", "microrregiao.mesorregiao.id": "ID"}, inplace=True)

In [7]:
# Preparar parâmetros para definir as opções nos radio buttons da telinha
# Cada um deve seguir o formato: ("texto", numero)
regioes = []
for i in cabecalho["Mesorregião"].sort_values(ascending=True).unique():
    regioes.append((i + " (" + str(len(cabecalho[cabecalho["Mesorregião"] == i]))+")", int(cabecalho[cabecalho["Mesorregião"] == i]["ID"].unique())))

In [8]:
# Referência: https://python-course.eu/tkinter/radio-buttons-in-tkinter.php
from tkinter import *
from IPython.display import clear_output    
from plotly.express import choropleth_mapbox
root = Tk()
root.title("Germano")
root.resizable(0,0)
v = IntVar()
# Conjunto de opções do menu
tRegioes = tuple(regioes)

def ShowChoice():
    exibe_mapa(v.get())
Label(root, 
         text="Escolha a região:",
         justify = LEFT,
         padx = 20).pack()

# Gera cada uma das opções do menu
for regiao, val in tRegioes:
    Radiobutton(root, 
                   text=regiao,
                   padx = 20, 
                   variable=v, 
                   command=ShowChoice,
                   value=val).pack(anchor=W)


root.mainloop()